In [2]:
import random
import jieba
import re
from collections import defaultdict
from functools import reduce

def combine_variable(pattern): #连接表示变量的?x和?*x
    pattern_clean=[]
    i=0
    while i < len(pattern):
        if pattern[i]=='?' and pattern[i+1].isalpha() and (i<len(pattern)-1):
            pattern_clean.append(''.join([pattern[i],pattern[i+1]]))
            i+=2
        elif (i<len(pattern)-2) and pattern[i]=='?' and pattern[i+1]=='*' and pattern[i+2].isalpha():
            pattern_clean.append(''.join([pattern[i],pattern[i+1],pattern[i+2]]))
            i+=3
        else:
            pattern_clean.append(pattern[i])
            i+=1
    return pattern_clean

def remove_variable(sentence):  #将句子切分为包含非变量词组的列表
    sentence_combine=[]
    sentence_comp=[]
    for comp in combine_variable(list(jieba.cut(sentence))):
        if (not is_pattern_segment(comp)) and (not is_variable(comp)):
            sentence_comp +=[comp]
        else:
            sentence_combine.append(''.join(sentence_comp))
            sentence_comp=[]
        if '' in sentence_combine: sentence_combine.remove('')
    return sentence_combine

def is_variable(pat):  #判断是否是?开头的单一变量
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

fail=[True,None]   #返回所有多变量和对应值的元组列表
def pat_match_with_seg(pattern,saying):
    if not pattern or not saying: return []
    pat=pattern[0]
    if is_variable(pat):
        return [(pat,saying[0])]+pat_match_with_seg(pattern[1:],saying[1:])
    elif is_pattern_segment(pat):
        match,index=segment_match(pattern,saying)
        return [match]+pat_match_with_seg(pattern[1:],saying[index:])
    elif pat==saying[0]:
        return pat_match_with_seg(pattern[1:],saying[1:])
    else:
        return fail

def segment_match(pattern,saying):    #返回多变量和对应值的元组，以及对应值中单词的个数，从后往前找
    seg_pat,rest=pattern[0],pattern[1:]
    seg_pat=seg_pat.replace('?*','?')
    if not rest: 
        return (seg_pat,saying), len(saying)
    for i, token in enumerate(saying):
        if rest[0]==token and is_match(rest[1:],saying[(i+1):]):
            return (seg_pat,saying[:i]),i
    return (seg_pat,saying),len(saying)
def is_match(rest,saying):    #判断非变量部分是否相同
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] !=saying[0]:
        return False
    return is_match(rest[1:],saying[1:])
            
def pat_to_dict(patterns):   #将变量与取值生成字典
    return {k: ''.join(v) if isinstance(v,list) else v for k, v in patterns}
def substitute(rule, parsed_rules):    #替换变量部分
    if not rule: return []
    return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:], parsed_rules)

In [3]:
rule_responses={
    '?*x的理想是?*y': ['你觉得?y有什么意义呢？', '为什么?x想?y', '?x实现?y的理想后还想做什么','我也听?x说过?x想?y'],
    '我试图?*x但因为?*y放弃了': ['你觉得?x有什么意义呢？', '?y具体带来了什么麻烦？', '就因为?y你就要放弃了吗','?y对?x有这么大的影响吗？'],
    '与其?*x不如?*y': ['我觉得?x和?y不是一样的吗', '为什么你想?y', '?y吗，我觉得?x更好', '我也觉得显然?y更明智'],
    '?*x想?*y所以?*x每天?*z': ['?x每天?z吗？', '?y有什么好的呢？', '我也想?y今天开始我也?z'],
}

In [4]:
def get_response(saying):
    for key in rule_responses:
        if all(re.findall(x,saying) for x in remove_variable(key)):
            answer_pattern=combine_variable(list(jieba.cut(random.choice(rule_responses[key]))))
            key_pattern=combine_variable(list(jieba.cut(key)))
            saying_pattern=combine_variable(list(jieba.cut(saying)))
            return ''.join(substitute(answer_pattern,pat_to_dict(pat_match_with_seg(key_pattern,saying_pattern))))
    return 'System error'


In [6]:
get_response('鸣人的理想是成为火影')

'你觉得成为火影有什么意义呢？'

In [7]:
get_response('我试图去公园跑步但因为下雨放弃了')

'下雨对去公园跑步有这么大的影响吗？'

In [8]:
get_response('与其授人以鱼不如授人以渔')

'授人以渔吗，我觉得授人以鱼更好'

In [11]:
get_response('鸣人想成为火影所以鸣人每天都在练习螺旋丸')

'我也想成为火影今天开始我也都在练习螺旋丸'